---
layout: post
title: Sprint 5 Blog
description:  Leaderboard
type: issues 
comments: true
---

Group
The purpose of our group’s program is to create a multiplayer drawing and guessing game inspired by Scribble.io. This program allows players to take turns drawing while others guess the word or phrase being illustrated. The goal is to provide an entertaining and interactive way for people to engage in creative gameplay, fostering collaboration, quick thinking, and creativity.

Individual
My individual feature is primarily focused on making a leaderboard to showcase your scores. This feature allows people to see their highscores, low scores, and challenge themselves with default scores to earn a better score. The purpose of the feature is to also allow for a more gamelike experience for players.


# Leaderboard API Technical Analysis

## 1. List Requests, Use of Lists, Dictionaries, and Database
The API interacts with the database using HTTP methods to perform CRUD operations. Here are the endpoints and their respective methods:

- **GET** `/api/leaderboard`: Retrieve the leaderboard entries.
- **POST** `/api/leaderboard`: Add a new leaderboard entry.
- **PUT** `/api/leaderboard`: Update an existing leaderboard entry.
- **DELETE** `/api/leaderboard/<profile_name>/<drawing_name>`: Delete a specific leaderboard entry.

### Use of Lists and Dictionaries
In the API, we use lists to handle multiple leaderboard entries and dictionaries to represent individual entries. The database rows are converted to dictionaries for JSON responses.

#### Example:
- List (Rows): The **leaderboard_db** list is an example of how rows are represented in memory.
- Dictionary (Columns): Each entry in the list is a dictionary representing a row with columns **profile_name, drawing_name, and score**.



In [ ]:
leaderboard_db = [
    {
        "profile_name": "ArtMaster",
        "drawing_name": "Sunset Beach",
        "score": 95
    },
    {
        "profile_name": "PixelPro",
        "drawing_name": "Mountain Valley",
        "score": 88
    }
]

## 2. Formatting Response Data (JSON) from API into DOM
The API returns JSON responses which can be used to update the DOM in a web application. For example, **the GET /api/leaderboard** endpoint returns a JSON array of leaderboard entries.

#### Example:

In [ ]:
    @leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    try:
        # Query the database to get all leaderboard entries, ordered by score in descending order
        entries = LeaderboardEntry.query.order_by(LeaderboardEntry.score.desc()).all()
        # Convert each entry to a dictionary format and return as a JSON response with status code 200
        return jsonify([entry.read() for entry in entries]), 200
    except Exception as e:
        # Return an error message with status code 500 if an exception occurs
        return jsonify({"error": str(e)}), 500
    
    
#response
    
    [
    {
        "profile_name": "ArtMaster",
        "drawing_name": "Sunset Beach",
        "score": 95
    },
    {
        "profile_name": "PixelPro",
        "drawing_name": "Mountain Valley",
        "score": 88
    }
]                                                                 

## 3. Database Queries
The API uses SQLAlchemy to interact with the database. SQLAlchemy queries return Python lists representing rows of data.

#### Example:

In [ ]:
entries = LeaderboardEntry.query.order_by(LeaderboardEntry.score.desc()).all() #query the database to get all leaderboard entries, ordered by score in descending order

## 4. CRUD Methods in Class
The **LeaderboardEntry** class defines methods to perform CRUD operations on the database columns.

#### Example:


In [ ]:
def create(self): #add the entry to the database
    try:
        db.session.add(self)
        db.session.commit()
    except IntegrityError:
        db.session.rollback()
        return None
    return self

def read(self): #convert each entry to a dictionary format
    return {
        "profile_name": self.profile_name,
        "drawing_name": self.drawing_name,
        "score": self.score
    }
    
def update(self, data): #update the entry in the database
    for key, value in data.items():
        setattr(self, key, value)
    db.session.commit()
    return self

def delete(self): #delete the entry from the database
    db.session.delete(self)
    db.session.commit()

## 5. API Class
We use Flask's **Blueprint** to define the API routes. The **leaderboard_api** blueprint handles the **GET, POST, PUT, and DELETE** methods, allowing us to organize the API endpoints and their implementations.

#### Example:


In [ ]:
leaderboard_api = Blueprint('leaderboard_api', __name__)

@leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    # Implementation

@leaderboard_api.route('/api/leaderboard', methods=['POST'])
def add_leaderboard_entry():
    # Implementation

@leaderboard_api.route('/api/leaderboard', methods=['PUT'])
def update_leaderboard_entry():
    # Implementation

@leaderboard_api.route('/api/leaderboard/<profile_name>/<drawing_name>', methods=['DELETE'])
def delete_leaderboard_entry(profile_name, drawing_name):
    # Implementation

## 6. Method with Sequencing, Selection, and Iteration
The API class and methods are defined to handle different types of HTTP requests (GET, POST, PUT, DELETE). Each method in the class corresponds to a specific CRUD operation.

#### Code showing this:

In [ ]:
@leaderboard_api.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    try:
        entries = LeaderboardEntry.query.order_by(LeaderboardEntry.score.desc()).all()
        return jsonify([entry.read() for entry in entries]), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@leaderboard_api.route('/api/leaderboard', methods=['POST'])
def add_leaderboard_entry():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No data provided"}), 400

        entry = LeaderboardEntry(
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=int(data.get('score', 0))
        )
        
        if entry.create():
            return jsonify({"message": "Entry added successfully"}), 201
        return jsonify({"error": "Failed to add entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500
    
@leaderboard_api.route('/api/leaderboard', methods=['PUT'])
def update_leaderboard_entry():
    try:
        data = request.get_json() #get the data from the request
        if not data:
            return jsonify({"error": "No data provided"}), 400 #400 error

        existing_entry = LeaderboardEntry.query.filter_by( #check if an entry with the same profile name and drawing name already exists
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name')
        ).first()

        new_score = int(data.get('score', 0))

        if existing_entry: #if the entry already exists, update the score
            existing_entry.score = new_score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        entry = LeaderboardEntry( #if the entry does not exist, create a new entry
            profile_name=data.get('profile_name'),
            drawing_name=data.get('drawing_name'),
            score=new_score
        )
        
        if entry.create(): #add the new entry to the database
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e: 
        db.session.rollback()
        return jsonify({"error": str(e)}), 500 #500 error
    
@leaderboard_api.route('/api/leaderboard/<profile_name>/<drawing_name>', methods=['DELETE'])
def delete_leaderboard_entry(profile_name, drawing_name):
    try:
        entry = LeaderboardEntry.query.filter_by(
            profile_name=profile_name,
            drawing_name=drawing_name
        ).first()
        
        if not entry:
            return jsonify({"error": "Entry not found"}), 404
            
        entry.delete()
        return jsonify({"message": "Entry deleted successfully"}), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500

## 7. Parameters and Return Type
The body of the request contains JSON data with the following fields:

- profile_name: The name of the player.
- drawing_name: The name of the drawing.
- score: The score achieved.

#### Example:

In [ ]:
# Example JSON payload 
{
    "profile_name": "ArtMaster",
    "drawing_name": "Sunset Beach",
    "score": 95
}

#Response
{
    "message": "Entry added successfully"
}

#in case of an error
{
    "error": "No data provided"
}

## 8. Call to Algorithm Request
The code block to make a request involves using the Fetch API to interact with the backend API endpoints. Here is an example of how to make a **GET** request to fetch the leaderboard data



#### Example:

In [ ]:
async function fetchLeaderboard() {
    try {
        const response = await fetch(API_URL);c #fetch the leaderboard data from the API
        if (!response.ok) throw new Error('Failed to fetch leaderboard'); 
        const data = await response.json();  #parse the response as JSON
        displayLeaderboard(data);
    } catch (error) { #handle any errors that occur during the fetch operation
        console.error('Error:', error);  
        document.getElementById('leaderboard').innerHTML = 
            '<tr><td colspan="5" style="text-align: center;">Error loading leaderboard</td></tr>'; 
    }
}

function displayLeaderboard(data) {                                         
    const tbody = document.getElementById('leaderboard'); 
    tbody.innerHTML = '';

    if (!data || data.length === 0) { #check if there are no entries in the data
        tbody.innerHTML = '<tr><td colspan="5" style="text-align: center;">No entries yet</td></tr>'; #response 
        return; 
    }

    data.sort((a, b) => b.score - a.score) #sort the entries by score in descending order
        .forEach((entry, index) => { # iterate over each entry and display it in a table row
            const row = document.createElement('tr'); #create a new table row
            row.innerHTML = ` #data that will show
                <td>${index + 1}</td> 
                <td>${entry.profile_name}</td>
                <td>${entry.drawing_name}</td>
                <td>${entry.score}</td>
                <td> 
                    <button onclick="deleteEntry('${entry.profile_name}', '${entry.drawing_name}')"  #delete button to delete the entry
                            class="delete-btn">Delete</button>
                </td>
            `;
            tbody.appendChild(row); #append the row to the table body
        });
}


# The fetchLeaderboard function sends a **GET** request to the /api/leaderboard endpoint to retrieve the leaderboard data. 
# The Fetch API is used to make the request, and the response is handled asynchronously.